In [1]:
#Import all required Python libraries/modules
#Requests will be used for fetching data
import requests
#Pandas will be used for manipulating the data
import pandas as pd
#bs4 will be used for webscraping
import bs4
#Numpy will be used for arrays and computations
import numpy as np
#Time and Random will be used to create delays when webscraping a series of pages
from time import sleep
from random import randint
# sqlite3 for storage of the dataset
import sqlite3
from contextlib import closing
from io import StringIO

In [2]:
# Setup SQLite Database
database = 'Book_DB.db'
conn = sqlite3.connect(database)
c = conn.cursor()

In [3]:
datearray = np.arange('2008-12-29', '2020-01-06', np.timedelta64(7,'D'), dtype='datetime64')
#test array
print(datearray)

['2008-12-29' '2009-01-05' '2009-01-12' '2009-01-19' '2009-01-26'
 '2009-02-02' '2009-02-09' '2009-02-16' '2009-02-23' '2009-03-02'
 '2009-03-09' '2009-03-16' '2009-03-23' '2009-03-30' '2009-04-06'
 '2009-04-13' '2009-04-20' '2009-04-27' '2009-05-04' '2009-05-11'
 '2009-05-18' '2009-05-25' '2009-06-01' '2009-06-08' '2009-06-15'
 '2009-06-22' '2009-06-29' '2009-07-06' '2009-07-13' '2009-07-20'
 '2009-07-27' '2009-08-03' '2009-08-10' '2009-08-17' '2009-08-24'
 '2009-08-31' '2009-09-07' '2009-09-14' '2009-09-21' '2009-09-28'
 '2009-10-05' '2009-10-12' '2009-10-19' '2009-10-26' '2009-11-02'
 '2009-11-09' '2009-11-16' '2009-11-23' '2009-11-30' '2009-12-07'
 '2009-12-14' '2009-12-21' '2009-12-28' '2010-01-04' '2010-01-11'
 '2010-01-18' '2010-01-25' '2010-02-01' '2010-02-08' '2010-02-15'
 '2010-02-22' '2010-03-01' '2010-03-08' '2010-03-15' '2010-03-22'
 '2010-03-29' '2010-04-05' '2010-04-12' '2010-04-19' '2010-04-26'
 '2010-05-03' '2010-05-10' '2010-05-17' '2010-05-24' '2010-05-31'
 '2010-06-

**Note** - Completing the scrape below takes approximately 30 - 35 minutes.

In [4]:
# Loop through the links
for ds in datearray:
    try:
        # Initiate request
        source = ('https://www.publishersweekly.com/pw/nielsen/top100/' + (str(ds).replace("-","")) +'.html')
        header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        }
        data = requests.get(source, headers=header).text
        # Create soup
        soup = bs4.BeautifulSoup(data, 'lxml')
        # Isolate the block of data containing the table
        block = soup.find('div',class_='nielsen-wrapper')

        # Load the table
        df = pd.read_html(StringIO(data), header=0)

        # Create the DataFrame
        pwlist = pd.concat(df)

        # Drop empty columns
        pwlist = pwlist.dropna(axis=1, how='all')

        # Split data in Biographical Information column
        title = pwlist['Bibliographical Information  (Click title for review)'].str.split("  +",expand = True)
        # Select columns to keep and assign to a value
        title_keep = [0, 1]
        # Recreate DataFrame containing keep column(s)
        title = title[title_keep]
        # Rename columns for simplification
        title.columns = ['Title', 'Authors']

        # Split data in Authors column
        author = title['Authors'].str.split(",",expand = True)
        # Select columns to keep and assign to a value
        author_keep = [0]
        # Recreate DataFrame containing keep column(s)
        author = author[author_keep]
        # Rename column for simplification
        author.columns = ['Author']

        # Split data in Units YTD column
        units = pwlist['Units  2024 YTD'].str.split(' +', n=0, expand=True)
        # Rename colunms for simplification
        units.columns = ['Units Current', 'Units YTD']

        # Join all DataFrames
        pwlist = pd.concat([pwlist, title, author, units], axis=1)
        # Drop duplicate columns (from splits)
        pwlist.drop(columns=['Bibliographical Information  (Click title for review)', 'Units  2024 YTD', 'Authors'], inplace=True)
        # Add column with week identifier
        pwlist.insert(0, 'Week', ds, True)

        # Export to database
        pwlist.to_sql(name='PW_Bestsellers', con=conn, if_exists='append', index=False)

        #add delay to scrape interval
        sleep(randint(2,5))
    except Exception:
        pass

